# Black-Scholes and Implied Volatility

In [1]:
import math
from scipy.stats import norm

## Black-Scholes

In [2]:
def BlackScholes(S, K, T, r, q, sigma, price_type = 'c'):
    
    d1 = (math.log(float(S) / K) + (r - q + (sigma ** 2) / 2.) * T)/(sigma * math.sqrt(T))
    d2 = d1 - sigma * math.sqrt(T)
    
    if price_type == 'c':
        return S * math.exp(-q * T) * norm.cdf(d1) - K * math.exp(-r * T) * norm.cdf(d2)
    else:
        return K * math.exp(-r * T) * norm.cdf(-d2) - S * math.exp(-q * T) * norm.cdf(-d1)

#### Test:

In [7]:
BlackScholes(S = 100, K = 100, T = 1, r = 0.04, q = 0.02, sigma = 0.2, price_type = 'c')

8.7394879116103681

## Implied Volatility
#### Code in C++:

    double impliedVol(double spot, double mat, double strike, double price) {		//Brent's Method

        double tol = 0.000001;

        const int ITMAX = 100; 					//Maximum allowed number of iterations.
        const double EPS = std::numeric_limits<double>::epsilon();	//Machine floating-point precision.
        double a = 0.00001, b = 3, c = 3, d, e;
        double fa = BlackScholes(spot, a, mat, strike).Price() - price;
        double fb = BlackScholes(spot, b, mat, strike).Price() - price;
        double fc, p, q, r, s, xm, tol1;

        if ((fa > 0.0 && fb > 0.0) || (fa < 0.0 && fb < 0.0))
            throw("The root is not bracketed");
            //return NULL;

        fc = fb;

        for (int i = 0; i < ITMAX; ++i) {
            if ((fb > 0.0 && fc > 0.0) || (fb < 0.0 && fc < 0.0)) {
                c = a;			//Rename a, b, c and adjust bounding interval
                fc = fa;
                e = d = b - a;
            }
            if (abs(fc) < abs(fb)) {
                a = b;	fa = fb;
                b = c;	fb = fc;
                c = a;	fc = fa;
            }

            tol1 = 2.0 * EPS * abs(b) + 0.5*tol; //Convergence check.
            xm = 0.5 * (c - b);

            if (abs(xm) <= tol1 || fb == 0.0) return b;

            if (abs(e) >= tol1 && abs(fa) > abs(fb)) {
                s = fb / fa; 						//Attempt inverse quadratic interpolation.
                if (a == c) {
                    p = 2.0 * xm * s;
                    q = 1.0 - s;
                } else {
                    q = fa / fc;
                    r = fb / fc;
                    p = s * (2.0 * xm * q * (q - r) - (b - a) * (r - 1.0));
                    q = (q - 1.0) * (r - 1.0) * (s - 1.0);
                }
                if (p > 0.0) q = -q;
                p = abs(p);

                double min1 = 3.0 * xm * q - abs(tol1 * q);
                double min2 = abs(e * q);

                if (2.0*p < (min1 < min2 ? min1 : min2)) {
                    e = d; 					//Accept interpolation.
                    d = p / q;
                } else {
                    d = xm; 				//Interpolation failed, use bisection.
                    e = d;
                }
            } else { 					//Bounds decreasing too slowly, use bisection.
                d = xm;
                e = d;
            }

            a = b;	 					//Move last best guess to a.
            fa = fb;

            if (abs(d) > tol1) 			//Evaluate new trial root.
                b += d;
            else		
                b += (xm > 0) ? tol1 : -tol1;
                fb = BlackScholes(spot, b, mat, strike).Price() - price;
        }
        throw("Maximum number of iterations exceeded");
    }

#### Implementation in Python:

In [13]:
def impliedVol(spot, mat, strike, price, preset_r = 0, preset_q = 0, p_type = 'c'):
    # Brent's Method

	tol = 0.000001

	it_max = 100 					#Maximum allowed number of iterations.
	eps = 10 ** (-32)            	#Machine floating-point precision.
	a = 0.00001; b = 3; c = 3; d = 0; e = 0
	fa = BlackScholes(S = spot, K = strike, T = mat, r = preset_r, q = preset_q, sigma = a, price_type = p_type) - price
	fb = BlackScholes(S = spot, K = strike, T = mat, r = preset_r, q = preset_q, sigma = b, price_type = p_type) - price

	if ((fa > 0 and fb > 0) or (fa < 0 and fb < 0)):
		print("The root is not bracketed")
		#return None

	fc = fb

	for i in range (0, it_max + 1):
		if ((fb > 0 and fc > 0) or (fb < 0 and fc < 0)):
			c = a; fc = fa  # Rename a, b, c and adjust bounding interval
			d = b - a
			e = d
		if (abs(fc) < abs(fb)):
			a = b;	fa = fb
			b = c;	fb = fc
			c = a;	fc = fa

		tol1 = 2.0 * eps * abs(b) + 0.5 * tol  # Convergence check.
		xm = 0.5 * (c - b)

		if (abs(xm) <= tol1 or fb == 0): return b

		if (abs(e) >= tol1 and abs(fa) > abs(fb)):            
			s = fb / fa						#Attempt inverse quadratic interpolation.
			if (a == c):
				p = 2.0 * xm * s
				q = 1.0 - s
			else:
				q = fa / fc
				r = fb / fc
				p = s * (2.0 * xm * q * (q - r) - (b - a) * (r - 1.0))
				q = (q - 1.0) * (r - 1.0) * (s - 1.0)
                
			if (p > 0): q = -q
			p = abs(p)

			min1 = 3.0 * xm * q - abs(tol1 * q)
			min2 = abs(e * q)

			if (2.0 * p < (min1 if min1 < min2 else min2)):
				e = d 					# Accept interpolation.
				d = p / q
			else:
				d = xm 				# Interpolation failed, use bisection.
				e = d
		else: 					# Bounds decreasing too slowly, use bisection.
			d = xm
			e = d

		a = b	 					# Move last best guess to a.
		fa = fb

		if (abs(d) > tol1): 			#Evaluate new trial root.
			b += d
		else:
			b = b + tol1 if xm > 0 else b - tol1
			fb = BlackScholes(S = spot, K = strike, T = mat, r = preset_r, q = preset_q, sigma = b, price_type = p_type) - price
	print("Maximum number of iterations exceeded")

#### Test:

In [14]:
impliedVol(spot = 100, strike = 100, mat = 1, price = 8.7394879116103681, preset_r = 0.04, preset_q = 0.02, p_type = 'c')

2.9999993432539851